In [ ]:
import os, re
import pandas as pd
import altair as alt

from py_rhyme import PyRhyme


# Simulation object (Storing path to files)
simulation = {}
simulation['prefix'] = ##simulation_directory##
simulation['files'] = sorted(os.listdir(simulation['prefix']))


# PyRhyme Object
rhyme = PyRhyme(simulation['prefix'] + '/' + simulation['files'][0])
nsnaps = rhyme.ds.num_of_snapshots()


# data structure
data_structure = {
    'analysis': {
        'ifront': {
            'data': { 't': [], 'x': [], 'v': [], 'lines': {}, 'units': {
                't': 'Myr', 'x': 'kpc', 'v': 'km s^-1'
            }},
            'pos': lambda: ifront_pos('./analysis/ifront'),
            'vel': lambda: ifront_pos('./analysis/ifront'),
        }
    }
}

ifront = data_structure['analysis']['ifront']['data']


for root, sub_dirs in data_structure.items():
    if not os.path.exists(root):
        os.makedirs(root)
        
    for sub_dir in sub_dirs.keys():
        if not os.path.exists(root + '/' + sub_dir):
            os.makedirs(root + '/' + sub_dir)

# Ionization Front
Locating I-front and storing it into `data_structure['analysis']['ifront']['data']` object.

In [ ]:
v_conv = 3.086e+19 / 3.154e+13

p1 = [0, .5, .5]
p2 = [1, .5, .5]

if not (ifront['t'] and ifront['x'] and ifront['v']):
    for i in range(1, nsnaps):
        rhyme.ds.jump_to(i)
        ifront['lines'][i] = line = rhyme.lineout(p1, p2, 'ntr_frac_0.', n_intervals=128)
        ifront['t'].append(rhyme.ds.time())
        ifront['x'].append(line['x'][line['y'].index(min(line['y']))] * 1000)
        if i == 1:
            ifront['v'].append(ifront['x'][-1] / ifront['t'][-1])
        else:
            ifront['v'].append((ifront['x'][-1] - ifront['x'][-2]) / (ifront['t'][-1] - ifront['t'][-2]))
         

# Ionization Front Position as a function of Time

In [ ]:
ifront_position = pd.DataFrame({'t': ifront['t'], 'x': ifront['x']})

alt.Chart(ifront_position).mark_line(color='black').encode(
    alt.X('t', scale=alt.Scale(domain=(-3, 50), clamp=True)),
    alt.Y('x', scale=alt.Scale(domain=(4, 5.8), clamp=True))
)

Ionization Front Velocity as a function of Time

In [ ]:
ifront_velocity = pd.DataFrame({'t': ifront['t'][10:], 'v': ifront['v'][10:]})
    
alt.Chart(ifront_velocity).mark_point(color='black').encode(
    alt.X('t', scale=alt.Scale(domain=(-3, 50), clamp=True)),
    alt.Y('v', scale=alt.Scale(domain=(-1, 3), clamp=True))
)